In [5]:
%matplotlib inline

In [6]:
import orekit
vm = orekit.initVM()
import warnings
warnings.filterwarnings('ignore')
import ipydatetime
import ipywidgets as widgets
from IPython.display import display, clear_output

In [7]:
from orekit.pyhelpers import setup_orekit_curdir
setup_orekit_curdir()
orekit.pyhelpers.download_orekit_data_curdir()

In [8]:
from org.orekit.frames import FramesFactory, TopocentricFrame
from org.orekit.bodies import OneAxisEllipsoid, GeodeticPoint
from org.orekit.time import TimeScalesFactory, AbsoluteDate, DateComponents, TimeComponents
from org.orekit.utils import IERSConventions, Constants
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator

from java.util import Arrays
from orekit import JArray_double
from org.orekit.bodies import  OneAxisEllipsoid, GeodeticPoint
from org.orekit.frames import  FramesFactory, TopocentricFrame
from org.orekit.data import DataProvidersManager, ZipJarCrawler
from org.orekit.time import TimeScalesFactory, AbsoluteDate, DateComponents, TimeComponents
from org.orekit.orbits import KeplerianOrbit, CartesianOrbit
from org.orekit.utils import Constants
from org.orekit.propagation.analytical import EcksteinHechlerPropagator, KeplerianPropagator
from org.orekit.propagation.analytical.tle import TLE, TLEPropagator
from org.orekit.propagation.conversion import FiniteDifferencePropagatorConverter
from org.orekit.propagation.conversion import TLEPropagatorBuilder
from datetime import datetime
from org.orekit.propagation import SpacecraftState
from org.orekit.orbits import OrbitType, PositionAngle
from org.orekit.propagation.numerical import NumericalPropagator
from org.hipparchus.ode.nonstiff import DormandPrince853Integrator
from org.hipparchus.ode.nonstiff import ClassicalRungeKuttaIntegrator
from org.hipparchus.geometry.euclidean.threed import Vector3D
from org.orekit.forces.gravity.potential import GravityFieldFactory
from org.orekit.forces.gravity import HolmesFeatherstoneAttractionModel
from org.orekit.utils import IERSConventions, PVCoordinates, Constants
from java.io import File

#%% imports for matplotlib, math, and numpy

import matplotlib.dates as mdates
from math import radians, pi
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
#%% Some constants, ae, mu, utc
ae = Constants.WGS84_EARTH_EQUATORIAL_RADIUS
mu = Constants.WGS84_EARTH_MU
utc = TimeScalesFactory.getUTC()

In [9]:
j2000 = FramesFactory.getEME2000()
earth_frame = FramesFactory.getITRF(IERSConventions.IERS_2010, True)
utc = TimeScalesFactory.getUTC()

In [10]:
frame = widgets.RadioButtons(
    options=['J2000', 'ITRF'],
    value='ITRF',
    layout={'width': 'max-content'}, # If the items' names are long
    description='СК:',
    disabled=False
)
tle_line1 = widgets.Text(placeholder='TLE line 1', layout={'width': '95%'})
tle_line2 = widgets.Text(placeholder='TLE line 2', layout={'width': '95%'})
x = widgets.Text(placeholder='X', description='Км')
y = widgets.Text(placeholder='Y', description='Км')
z = widgets.Text(placeholder='Z', description='Км')
vx = widgets.Text(placeholder='Vx', description='Км/с')
vy = widgets.Text(placeholder='Vy', description='Км/с')
vz = widgets.Text(placeholder='Vz', description='Км/с')
mass = widgets.Text(placeholder='масса', value = '100', description='Кг')
s_time = widgets.Text(placeholder='Время', value = '900', description='Cек')
date = widgets.DatePicker(description='Дата',disabled=False)
time = ipydatetime.TimePicker(description = 'Время', step=0.001)
step_time = widgets.Text(placeholder='Step time (secs)')
button_send = widgets.Button(
                description='Расчитать TLE',
                tooltip='Send',
                style={'description_width': 'initial'},
                layout={'width': '50%'}
            )

output = widgets.Output()
def on_button_clicked(event):

    d = datetime.combine(date.value, time.value)
    sf = d.strftime('%S.%f')
    epochDate = AbsoluteDate(d.year, d.month, d.day, d.hour, d.minute, float(sf), utc)
    step_time = int(s_time.value) #sec
    satellite_mass = float(mass.value) # kg
    mytle = TLE(tle_line1.value,tle_line2.value)

    p = Vector3D(float(x.value) *1000,float(y.value) *1000,float(z.value) *1000)
    v = Vector3D(float(vx.value) *1000,float(vy.value) *1000,float(vz.value) *1000)
    pv = PVCoordinates(p, v)
    j2000 = FramesFactory.getEME2000()
    gtod_frame = FramesFactory.getGTOD(IERSConventions.IERS_2010, True)
    
    if frame.value == 'J2000':
        initialOrbit = KeplerianOrbit(pv, j2000, epochDate, mu)
    else:
        tp = gtod_frame.getTransformTo(j2000, epochDate).transformPVCoordinates(pv).getPosition()
        tv = gtod_frame.getTransformTo(j2000, epochDate).transformPVCoordinates(pv).getVelocity()
        tpv = PVCoordinates(tp,tv)
        initialOrbit = KeplerianOrbit(tpv, j2000, epochDate, mu)
    
    itrf    = FramesFactory.getITRF(IERSConventions.IERS_2010, True) # International Terrestrial Reference Frame, earth fixed
    earth = OneAxisEllipsoid(Constants.WGS84_EARTH_EQUATORIAL_RADIUS,
                             Constants.WGS84_EARTH_FLATTENING,
                             itrf)
    initialState = SpacecraftState(initialOrbit, satellite_mass)
    initialDate = epochDate
    integrator = ClassicalRungeKuttaIntegrator(30.0)
    propagator = NumericalPropagator(integrator)
    gravityProvider = GravityFieldFactory.getNormalizedProvider(8, 8)
    propagator.addForceModel(HolmesFeatherstoneAttractionModel(earth.getBodyFrame(), gravityProvider))
    startDate = epochDate
    dt = startDate.shiftedBy(float(step_time))
    propagator.setInitialState(initialState)
    state = propagator.propagate(dt)
    newpv = state.getPVCoordinates()
    dtp = j2000.getTransformTo(gtod_frame, dt).transformPVCoordinates(newpv).getPosition()
    dtv = j2000.getTransformTo(gtod_frame, dt).transformPVCoordinates(newpv).getVelocity()
    dtpv = PVCoordinates(dtp, dtv)
    neworbit = KeplerianOrbit(newpv, j2000, dt, mu)
    newtle = TLE.stateToTLE(state, mytle)
    mtle = TLE.stateToTLE(initialState, mytle)
    with output:
        clear_output()
        print('')
        print('Получившаяся TLE')
        print(newtle)
        print('')
        print('Получившаяся орбита')
        print(neworbit)
        print('')
        print('Исходная орбита')
        print(initialOrbit)
button_send.on_click(on_button_clicked)

vbox_result = widgets.VBox([button_send, output])

text_0 = widgets.HTML(value="<h1>Перевод вектора в TLE</h1>")
text_1 = widgets.HTML(value="<h2>Вектор состояния</h2>")
text_2= widgets.HTML(value="<h2>Вектор скорости </h2>")
text_3= widgets.HTML(value="<h2>Шаблонная TLE </h2>")
text_4 = widgets.HTML(value="<h2> На сколько продвигать? </h2>")
text_5 = widgets.HTML(value="<h2> Масса </h2>")
text_6 = widgets.HTML(value="<h2> Система координат </h2>")
text_7 = widgets.HTML(value="<h2> Время (UTC) </h2>")
vbox_text = widgets.VBox([text_0, text_6, frame, text_7, date, time, text_1, x, y, z, text_2, vx, vy, vz, text_3, tle_line1, tle_line2, text_4, s_time, text_5, mass, vbox_result])
page = widgets.HBox([vbox_text])
display(page)